<center>
<h1 dir=rtl>
«تمرین برنامه‌نویسی اول»
</h1>
<h3 dir=rtl>
 نصب اسپارک و پردازش‌های اولیه با آن
</h3>
</center>

<div dir=rtl>
ابتدا بسته ‌اسپارک را در پایتون نصب کنید:
</div>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<div dir=rtl>
سپس کتابخانه‌های مربوطه را import کنید.
</div>

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

<div dir=rtl>
با دستورات ذیل یک session و context برای اسپارک ایجاد می‌کنیم. 
</div>

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

<div dir=rtl>
دستور spark را اجرا کنید تا مطمئن شوید که همه‌چیز به درستی نصب و تنظیم شده است
</div>

In [ ]:
spark

<div dir=rtl>
در این تمرین یک دیتاست معروف به نام Record Linkage را دانلود می‌کنیم. 

سوال اول: از لینک ذیل درباره این دیتاست مطالعه کنید و در یک پاراگراف آنرا معرفی کنید.
داده های فوق دارای 5749132 ثبت و 12 صفت است. داده ها در سال 2011 جمع آوری شده که شامل اطلاعات شخصی افراد مریوط به سال 2005 تا 2008 است.جفت‌های داده‌ها در طی یک بررسی دستی گسترده که در آن چندین مستند درگیر بود، به‌عنوان «مطابقت» یا «غیر منطبق» طبقه‌بندی شدند.
</div>
<div>
<a href="https://archive.ics.uci.edu/ml/datasets/record+linkage+comparison+patterns">https://archive.ics.uci.edu/ml/datasets/record+linkage+comparison+patterns</a>
</div>

In [ ]:
!curl -L -o donation.zip https://bit.ly/1Aoywaq
!unzip donation.zip
!unzip 'block_*.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   1752      0 --:--:-- --:--:-- --:--:--  1752
100 53.8M  100 53.8M    0     0  56.1M      0 --:--:-- --:--:-- --:--:-- 81.7M
Archive:  donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         
Archive:  block_6.zip
  inflating: block_6.csv             

Archive:  block_4.zip
  inflating: block_4.csv             

Archive:  block_7.zip
  inflating: block_7.csv             

Archive:  block_5.zip


<div dir=rtl>
در ادامه، همه فایل‌های csv مربوط به این دیتاست را در یک دیتافریم لود می‌کنیم:
</div>

In [ ]:
df0 = spark.read.csv("block*.csv")

<div dir=rtl>
با دستور ذیل تعداد رکوردهای این دیتافریم را بدست می‌آوریم:
</div>

In [ ]:
df0.count()

5749142

<div dir=rtl>
با این دستور شمای این دیتافریم را مشاهده می‌کنیم. دقت کنید که نام ستونها و نوع آنها همگی به صورت خودکار تولید شده‌اند
</div>

In [ ]:
df0.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



<div dir=rtl>
برای نمایش ۵ سطر اول این دیتافریم از دستور ذیل استفاده می‌کنیم
</div>

In [ ]:
df0.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

<div dir=rtl>
حال، مجدد فایل‌های csv را با پارامترهای دیگری لود می‌کنیم:
</div>

In [ ]:
parsed = spark.read.csv("block*.csv", header=True, inferSchema=True, nullValue="?")

In [ ]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [ ]:
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

<div dir=rtl>
سوال دوم:
با تغییر انجام شده در نحوه خواندن فایل‌های csv چه تغییری در DataFrame جدید نسبت به حالت قبل ایجاد شده است؟
</div>


در این قسمت عنوان ها به شکل داده نیستند و به صورت عنوان داده ظاهر شده اند همچنین داده های که پر نشده بودند با تهی جایگزین شده اند.

In [ ]:
parsed.groupBy('is_match').count().show()

+--------+-------+
|is_match|  count|
+--------+-------+
|    true|  20931|
|   false|5728201|
+--------+-------+



<div dir=rtl>
سوال سوم: 
دستور بالا چه کاری انجام می‌دهد؟
</div>

دستور فوق داده هایی که ماطبقت داشته و داده های غیر منطبق را شمارش کرده است.یعنی 20931 مطابقت دارند.

<div dir=rtl>
برای نمایش اطلاعات آماری از دیتاست از دستور ذیل استفاده می‌کنیم:
</div>

In [ ]:
summary = parsed.describe()
summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|           5748125|            103698|            5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704425707| 0.900017671890335|0.3156278193076

<div dir=rtl>
سوال چهارم:
چرا مقادیر count در ستون‌های مختلف متفاوت است؟
</div>

به دلیل داشتن داده های null.

<div dir=rtl>
می‌توان آماره‌های مد نظر را روی دو دسته رکوردهایی که مقادیر فیلد is_matched آنها متفاوت است اجرا کرد. 

در اینجا به نحوه  فیلتر  بر اساس این فیلد توجه کنید.
</div>

In [ ]:
matches = parsed.where("is_match = true")
match_summary = matches.describe()

misses = parsed.filter(col("is_match") == False) 
miss_summary = misses.describe()

<div dir=rtl>
روی دیتافریم ایجاد شده می‌توان انواع پیش‌پردازش‌ها را انجام داد. 
به عنوان نمونه فرض کنید می‌خواهیم تمام مقادیر null را با مقدار صفر جایگزین کنیم.

سوال پنجم: برای انجام این کار از چه دستوری باید استفاده کرد؟

</div>

ابتدا با دستور


parsed[parsed.isnull().any(axis=1)]
موقعیت ثبت های تهی را شناسایی میکنیم و سپس با دستور 

parsed.replace()
استفاده میکنیم.


یا از دستور parsed.na.fill(0) استفاده نماییم.

In [ ]:
# Fill the null values  with 0
